# 12. Strategy Report Blueprint (Single Symbol)

> ⚠️ Research tutorial only. Not financial advice.

このノートは、1銘柄向けに**日次レポート1枚**を生成する練習です。
将来の iPhone ダッシュボードに載せるための雛形（blueprint）を意識します。

## 1) レポート項目（設計）

- Regime stats（相場状態ごとの成績）
- Signal frequency（どれくらい売買シグナルが出るか）
- Performance summary（累積・年率・DD）
- Drawdown curve（どこで苦しむか）
- When it fails（ワースト取引周辺の可視化）

In [ ]:
import sys
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

repo_root = Path.cwd()
if not (repo_root / 'src').exists():
    repo_root = Path.cwd().parent
sys.path.insert(0, str(repo_root / 'src'))

from quantlab.data import fetch_ohlc
from quantlab.indicators import atr, ema

pd.set_option('display.float_format', lambda x: f'{x:,.4f}')
plt.style.use('seaborn-v0_8')

In [ ]:
# --- knobs ---
SYMBOL = 'SPY'
PERIOD = '5y'
INTERVAL = '1d'
EMA_FAST, EMA_SLOW = 12, 26
ATR_N = 14
FEE_BPS, SLIPPAGE_BPS = 2.0, 3.0

try:
    df = fetch_ohlc(SYMBOL, period=PERIOD, interval=INTERVAL).copy()
except Exception as e:
    print(f'Fetch failed ({e}); using synthetic data.')
    idx = pd.date_range('2020-01-01', periods=900, freq='B')
    rnd = np.random.default_rng(7).normal(0.0002, 0.011, size=len(idx))
    close = 100 * (1 + pd.Series(rnd, index=idx)).cumprod()
    df = pd.DataFrame(index=idx)
    df['Close'] = close
    df['Open'] = df['Close'].shift(1).fillna(df['Close'])
    df['High'] = np.maximum(df['Open'], df['Close']) * 1.003
    df['Low'] = np.minimum(df['Open'], df['Close']) * 0.997
    df['Volume'] = 1_000_000

df.head()

## 2) シグナル、リターン、ドローダウン計算

In [ ]:
# 方向シグナル + ATR レジーム
df['ema_fast'] = ema(df['Close'], EMA_FAST)
df['ema_slow'] = ema(df['Close'], EMA_SLOW)
df['signal'] = (df['ema_fast'] > df['ema_slow']).astype(float)
df['ret'] = df['Close'].pct_change().fillna(0.0)
df['atr'] = atr(df, ATR_N)
df['atr_ratio'] = (df['atr'] / df['Close']).replace(0, np.nan)

# 研究段階の単純戦略: signal をそのまま重みとして利用
df['w'] = df['signal']
df['dw'] = df['w'].diff().abs().fillna(0.0)
df['gross_ret'] = df['w'].shift(1).fillna(0.0) * df['ret']
df['cost'] = ((FEE_BPS + SLIPPAGE_BPS)/10_000.0) * df['dw']
df['net_ret'] = df['gross_ret'] - df['cost']
df['equity'] = (1 + df['net_ret']).cumprod()
df['drawdown'] = df['equity'] / df['equity'].cummax() - 1

# トレンドとボラのレジーム分類（教育用の粗い区分）
df['trend_regime'] = np.where(df['ema_fast'] >= df['ema_slow'], 'bull', 'bear')
q1 = df['atr_ratio'].quantile(0.33)
q2 = df['atr_ratio'].quantile(0.66)
df['vol_regime'] = pd.cut(df['atr_ratio'], bins=[-np.inf, q1, q2, np.inf], labels=['low', 'mid', 'high'])

df[['Close','signal','net_ret','equity','drawdown','trend_regime','vol_regime']].tail()

## 3) Regime stats と Signal frequency

In [ ]:
# レジーム別の平均リターン/ボラ/件数
regime_stats = (
    df.dropna(subset=['vol_regime'])
      .groupby(['trend_regime', 'vol_regime'])['net_ret']
      .agg(['count', 'mean', 'std'])
      .rename(columns={'count':'n_days', 'mean':'avg_daily_ret', 'std':'daily_vol'})
)

# シグナル頻度（売買の切替回数）
signal_changes = df['signal'].diff().abs().fillna(0.0)
signal_switches = int((signal_changes > 0).sum())
switches_20d = signal_changes.tail(20).sum()

print(f'Signal switches (all period): {signal_switches}')
print(f'Signal switch intensity (last 20d): {switches_20d:.1f}')
regime_stats

## 4) Performance summary（単一テーブル）

In [ ]:
ann = 252
ann_ret = (1 + df['net_ret']).prod() ** (ann / len(df)) - 1
ann_vol = df['net_ret'].std() * np.sqrt(ann)
max_dd = df['drawdown'].min()
win_rate = (df['net_ret'] > 0).mean()

summary = pd.Series({
    'Symbol': SYMBOL,
    'Days': len(df),
    'Cumulative Return (net)': df['equity'].iloc[-1] - 1,
    'Annual Return (net)': ann_ret,
    'Annual Vol (net)': ann_vol,
    'Max Drawdown': max_dd,
    'Win Rate (daily)': win_rate,
    'Turnover Sum': df['dw'].sum(),
})
summary

## 5) Drawdown curve

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))
ax.plot(df.index, df['drawdown'], color='tab:red', linewidth=1.5)
ax.set_title(f'{SYMBOL} Drawdown Curve (Net)')
ax.set_ylabel('Drawdown')
ax.grid(True, alpha=0.3)
plt.show()

### What to observe

- DD が急に深くなる時期は、シグナル方向と相場レジームが噛み合っていない可能性があります。
- レジーム別 stats と重ねて、どの環境で脆いかを確認してください。

## 6) When it fails: ワースト日と周辺ウィンドウ

In [ ]:
# 最悪の日次リターンを3つ抽出し、前後5営業日を可視化
worst_days = df.nsmallest(3, 'net_ret').index
window = 5

print('Worst days:')
for d in worst_days:
    print(d.date(), f"net_ret={df.loc[d, 'net_ret']:.4%}", f"drawdown={df.loc[d, 'drawdown']:.2%}")

fig, axes = plt.subplots(len(worst_days), 1, figsize=(12, 3*len(worst_days)), sharex=False)
if len(worst_days) == 1:
    axes = [axes]

for ax, d in zip(axes, worst_days):
    i = df.index.get_loc(d)
    lo, hi = max(0, i-window), min(len(df)-1, i+window)
    w = df.iloc[lo:hi+1]
    ax.plot(w.index, w['Close'], label='Close', color='tab:blue')
    ax2 = ax.twinx()
    ax2.step(w.index, w['w'], where='post', label='Position', color='tab:orange', alpha=0.7)
    ax.axvline(d, color='tab:red', linestyle='--', alpha=0.8)
    ax.set_title(f'Failure Window around {d.date()}')
    ax.grid(True, alpha=0.25)
plt.tight_layout()
plt.show()

## 7) iPhone dashboard 向けの最小出力イメージ

このノートの集計値は `docs/05_iphone_integration_plan.md` の JSON 契約へ接続できます。
次段階では、ここで計算した summary/regime/signal を JSON に整形し、
配信経路（iCloud / API / GitHub Raw）へ流す設計に進みます。